## Fitting a Cluster Expansion Model

In order to predict thermodynamic properties of a material system, it is necessary to calculate energies of many millions of configurational microstates. This is currently intractable with existing high-accuracy methods (such as Density Functional Theory or DFT). 

The cluster expansion model is a computationally inexpensive surrogate model that approximates DFT predictions. A cluster expansion model must be trained on a set of high accuracy data, and can then be used to predict configurational energies. 

A cluster expansion follows the form y=Xv, where y is a vector of n formation energies, X is a matrix of correlation vectors (basis vectors) stored as row vectors, and v is a vector of k effective cluster interactions (ECI), which serve as coefficients of a cluster expansion model. 

The model is then defined by the choice of basis vectors (columns) in X, as well as the values for the ECI in v.   


A "good" cluster expansion will have these characteristics:

### 1) Low predictive error (low root mean squared error , or RMSE):

A low predictive error is always a basic requirement of any model. Predictions that are far from observed values are not desirable. An ideal rmse for many systems is around 1-5 meV per atom. 

### 2) Sparsity 

The cluster expansion model is intended to be computationally indexpensive. Adding additional basis vectors to the model can imporove the model accuracy, but will also slow the model. Including too many basis vectors can also lead to overfitting, which is also problematic. An ideal cluster expansion has the minimal number of basis vectors to make accurate predictions; this is considered a sparse model. 

### 3) High ground state accuracy: No missing ground states, no spurious ground states. 

The ground states of a system, observed through DFT or through experiment, is thermodynamic data. These ground states dictate qualitative aspects of other thermodynamic data, such as phase diagrams or voltage curves. Therefore, an ideal model has high ground state replication. 

### 4) Uncertainty informaiton 
Rudimentary regression methods produce single models. For transparency, it is desirable to produce a distribution of models, rather than a single model.  

In [121]:
#Load data: remake later

import numpy as np
import thermocore.io.casm as cio 
import json
import thermocore.geometry.hull as thull
from bokeh.plotting import figure, show, output_file
from sklearn.metrics import mean_squared_error

# Json dictionary file in the form of {"eci":[]} . Full dimensionality, un-used eci are set exactly to 0
with open("/media/derick/big_storage/research_backup/DeoResearch/experiments/pancake4_min_rmse_scaling/anchor_eci.json", "r") as f:
    anchor_eci = np.array(json.load(f)["eci"])

# get non zero anchor eci indices
#nonzero_indices = np.nonzero(anchor_eci)[0]
upscaling_vector = ~np.isclose(anchor_eci, 0, atol=1e-5)
print(np.count_nonzero(upscaling_vector))
print(upscaling_vector.shape)


# Load calculated configurations query data
with open("/media/derick/big_storage/research_backup/DeoResearch/experiments/pancake4_min_rmse_scaling/ZrN_FCC_1.2.0_TITAN_calculated.json", "r") as f:
    calculated_query = json.load(f)
    calculated_data = cio.regroup_query_by_config_property(calculated_query)
names = np.array(calculated_data["name"])
corrs = np.array(calculated_data["corr"])
comps = np.array(calculated_data["comp"])
formation_energies = np.array(calculated_data["formation_energy"])






142
(1335,)


In [122]:
#Necessary Functions:
from bokeh.models import Label
from bokeh.plotting import figure, show, output_file
from sklearn.metrics import mean_squared_error
from bokeh.models import LinearAxis, Range1d
from bokeh.io import output_notebook
output_notebook()
from sklearn.linear_model import RidgeCV, BayesianRidge


def binary_convex_hull_plotter(comp, observed_energies, predicted_energies:np.ndarray=None):
    """Formation energy and convex hull plotter. 

    Parameters
    ----------
    comp: np.ndarray
        Matrix of compositons, shape (n,m) of n configurations, m composition dimensions. 
    observed_energies: np.ndarray 
        Vector of observed formation energies, shape (n,) of n configurations. 
    predicted_energies: np.ndarray, optional
        Vector of predicted formation energies, shape (n,) of n configurations. 

    Returns
    -------
    p:figurebokeh.plotting._figure.figure    
        Bokeh figure object. Can be displayed with bokeh.plotting.show(p)
    """

    # Calculate observed hull vertices
    observed_hull = thull.full_hull(compositions=comp, energies=observed_energies)
    observed_vertices, _ = thull.lower_hull(observed_hull)

    # Sort observed vertices
    hullcomps = np.ravel(comp[observed_vertices])
    sorted_obs_verts = observed_vertices[np.argsort(hullcomps)]

    # Extract hull components and energies
    sorted_obs_hullcomps = np.ravel(comp[sorted_obs_verts])
    sorted_obs_hulleng = observed_energies[sorted_obs_verts]

    # Create a blank figure

    p = figure(title="Observed Energies", x_axis_label="Composition", y_axis_label="Formation Energy", width=1100, height=800)


    if predicted_energies is not None:
        # Calculate predicted hull vertices
        predicted_hull = thull.full_hull(compositions=comp, energies=predicted_energies)
        predicted_vertices, _ = thull.lower_hull(predicted_hull)

        # Sort predicted vertices
        hullcomps = np.ravel(comp[predicted_vertices])
        sorted_pred_verts = predicted_vertices[np.argsort(hullcomps)]

        # Extract predicted hull components and energies
        sorted_pred_hullcomps = np.ravel(comp[sorted_pred_verts])
        sorted_pred_hulleng = predicted_energies[sorted_pred_verts]

        #Plot green squares on missing ground states 
        missing_gs = []
        for ov in observed_vertices:
            if ov not in predicted_vertices:
                missing_gs.append(ov)
        p.scatter(np.ravel(comp[missing_gs]), predicted_energies[missing_gs], marker='square', color='green', size=15, alpha=0.5, legend_label="Missing")

        #Plot blue squares on spurious ground states
        spurious_gs = []
        for pv in predicted_vertices:
            if pv not in observed_vertices:
                spurious_gs.append(pv)
        p.scatter(np.ravel(comp[spurious_gs]), predicted_energies[spurious_gs], color='blue', marker = 'square', size=15,alpha=0.5, legend_label="Spurious")        


    # Scatter plot for observed energies (in black)
    p.scatter(np.ravel(comp), observed_energies, size=8, color="black",marker='x', legend_label="Observed Energies")

    # Line plot for hull vertices (in black)
    p.scatter(sorted_obs_hullcomps, sorted_obs_hulleng,marker='diamond', size=15, color="black")
    p.line(sorted_obs_hullcomps, sorted_obs_hulleng, line_width=2, color="black", legend_label="Hull Vertices (Observed)")

    # Check if predicted energies are available
    if predicted_energies is not None:
    

        # Scatter plot for predicted energies (in red)
        p.scatter(np.ravel(comp), predicted_energies, size=8, color="red",marker="y", legend_label="Predicted Energies")

        # Line plot for predicted hull vertices (in red)
        p.scatter(sorted_pred_hullcomps, sorted_pred_hulleng, marker="diamond",size=8, color="red")
        p.line(sorted_pred_hullcomps, sorted_pred_hulleng, line_width=2, color="red", legend_label="Hull Vertices (Predicted)")

    
    return p


def eci_plot(eci:np.ndarray, basis_dict:dict=None):
    """Plots the values of ECI.
    Parameters
    ----------
    eci: np.ndarray
        ECI values.
    basis_dict:dict, optional
        basis dictionary from CASM. If provided, plot will divide ECI into pairs, triplets, etc.
    
    Returns
    -------
        p:figurebokeh.plotting._figure.figure    
        Bokeh figure object. Can be displayed with bokeh.plotting.show(p)
    """
    p = figure(title="ECI Values", x_axis_label="ECI Index", y_axis_label="ECI value", width=1100, height=800)
    p.scatter(list(range(len(eci))), eci, color='black')
    return p


def hulldist_weighting(
    corr: np.ndarray,
    formation_energy: np.ndarray,
    hulldists: np.ndarray,
    b: float = 1,
):
    """Given boltzmann weighting parameters, returns a weighted feature matrix and corresponding target vector.

    Parameters
    ----------
    B : float
        Boltzmann weighting coefficient (inverse temperature)

    Returns
    -------
    x_prime : numpy.ndarray
        Weighted CASM nxk correlation matrix, n = number of configurations, k = number of ECI.
    y_prime : numpy.ndarray
        Weighted n-dimensional vector of formation energies.
    """

    weight = np.identity(formation_energy.shape[0])
    for config_index in range(formation_energy.shape[0]):
        weight[config_index, config_index] = np.exp(-b*hulldists[config_index])
    l = np.linalg.cholesky(weight)
    y_prime = np.ravel(l @ formation_energy.reshape(-1, 1))
    x_prime = l @ corr
    return (x_prime, y_prime)    


Loading BokehJS ...

### Low predictive Error: Ordinary Least Squares

In the limit of infinite data, ordinary least squares will always produce the most accurate model within a given set of basis vectors. 

Assuming that your data is not under-determined (that is, assuming the number of data points is >= to the number of basis vectors), ordinary least squares will find the solution with the lowest RMSE within the current basis vector space. 

If there are more basis vectors than data points, ordinary least squares will probably overfit to your data. 


In [123]:
#Calcualte the Ordinary Least Squares solution
ols_solution = np.linalg.pinv(corrs.T @ corrs) @ corrs.T @ formation_energies


#Create the convex hull plot object using bokeh
plot = binary_convex_hull_plotter(comp=comps, observed_energies=formation_energies, predicted_energies=corrs@ols_solution)

#calculate the RMSE
print("OLS RMSE, eV/primitive cell: ", mean_squared_error(formation_energies, corrs@ols_solution, squared=False))
print("Nonzero ECI count: ", np.count_nonzero(ols_solution))

#Display the bokeh plot in browser
show(plot)


OLS RMSE, eV/primitive cell:  0.0020437891380188666
Nonzero ECI count:  1335


#### OLS overfitting
As warned above, the ordinary least squares solution is accurate, but if the basis vector space is larger than the number of calculated configurations, OLS will probably overfit. Plot the values of the ECI to check. 

If any ECI values  have a magnitude greater than 1 eV, there is likely some overfitting. 

In [124]:
#Check the ECI values of the OLS solution 
p = eci_plot(ols_solution)
show(p)

### Low Predictive Error: Ridge Regression

While ordinary least squares can be useful and accurate, it has a tendency to over-fit to data, especially when given a large basis vector space. During overfitting, it is common to see exremely large (unphysical) values for ECI. 

Regularized models prevent overfitting by placing a constraint on the overall magnitude of the solution. A common form of regularized linear regression is Ridge regression. Ridge regression solutions are often similar to ordinary least squares when there is no case of overfitting.

While Ridge generally produces accurate models, it cannot create sparse models on its own. 

Cross validated ridge regression is implemented in scikit learn as RidgeCV, and is demonstrated below. 

In [125]:
#Create regularization object 
ridge_object = RidgeCV(fit_intercept=False, alphas =np.logspace(-4,4))

#do an initial fit to determine appropriate regularizer order of magnitude. (RidgeCV calls the regularizer alpha)
ridge_object.fit(corrs, formation_energies)
test_alpha = ridge_object.alpha_
print("Regularizer, Coarse sampling: ",test_alpha)

#recreate the ridge object with a finer grid of possible regularizer values
ridge_object= RidgeCV(fit_intercept=False, alphas = np.linspace(0.1*test_alpha,2*test_alpha,500))
ridge_object.fit(corrs, formation_energies)
print("Regularizer, fine sampling: ",ridge_object.alpha_)


#Extract and visualize the solution
ridge_solution = ridge_object.coef_


print("RidgeCV RMSE, eV/primitive cell: ", mean_squared_error(formation_energies, corrs@ridge_solution, squared=False))
print("Nonzero ECI: ", np.count_nonzero(ridge_solution))

#Create the convex hull plot object using bokeh
plot = binary_convex_hull_plotter(comp=comps, observed_energies=formation_energies, predicted_energies=corrs@ridge_solution)



#Display the bokeh plot in browser
show(plot)




Regularizer, Coarse sampling:  0.0013894954943731374
Regularizer, fine sampling:  0.0014563249369882786
RidgeCV RMSE, eV/primitive cell:  0.0024124352132489876
Nonzero ECI:  1335


#### Preventing Overfitting with Regularization: Ridge 

If the number of basis vectors exceeds the number of calculated configurations, ridge will prevent overfitting.

Plot the Ridge ECI values and compare to the OLS ECI values. 

In [126]:
#Show the ECI values of the Ridge solution 
p = eci_plot(ridge_solution)
show(p)

### Sparsity: manual truncation with Ordinary Least Squares

Ordinary least squares produces a model with the lowest RMSE within the current basis vector space. It does not truncate the basis set. 

To train a rudimentary sparse model with ordinary least squares, we can create a selection vector that activates only a subset of possible basis vectors. Then, perform ordinary least squares regression.

(It is usually difficult to obtain quality models using this approach, because the relevant basis vectors are often unknown.)

In [127]:
#Manually reduce the number of basis vectors 
selected_basis_mask = np.zeros(corrs.shape[1]).astype(int)
selected_basis_mask[0:20] = 1
selected_basis_indices = np.where(selected_basis_mask != 0)[0]

#Calcualte the Ordinary Least Squares solution
ols_solution = np.linalg.pinv(corrs[:,selected_basis_indices].T @ corrs[:,selected_basis_indices]) @ corrs[:,selected_basis_indices].T @ formation_energies

#Create the convex hull plot object using bokeh
plot = binary_convex_hull_plotter(comp=comps, observed_energies=formation_energies, predicted_energies=corrs[:,selected_basis_indices]@ols_solution)

#Display the bokeh plot in browser
show(plot)



### Sparsity: LassoLarsCV

A common approach to creating sparse models is LASSO (Least Absolute Shrinkage and Selection Operator). Basic LASSO often truncates too aggressively, producing unnaceptably high errors, though you can try it using scikit learn. 

LassoLarsCV is a cross validated (CV) version of LASSO which also uses least angle regression (LARS). In practice, it has successfully produced sparse models with low error. 

It is useful for finding a relevant truncated basis vector space. 



In [128]:
from sklearn.linear_model import LassoLarsCV
from sklearn.metrics import mean_squared_error

llars_regression_object = LassoLarsCV(fit_intercept=False)
llars_regression_object.fit(corrs, formation_energies)
llars_solution = llars_regression_object.coef_

#Observe the number of nonzero ECI:
print("There are ",np.count_nonzero(llars_solution), " nonzero ECI, of ",llars_solution.shape[0]," total ECI")

nonzero_indices = np.where(llars_solution !=0)[0]

#Check the RMSE for the current model:
print("RMSE for LassoLars: ",mean_squared_error(formation_energies,corrs@llars_solution,squared=False), "eV per primitive cell")

#Create and display a plot of the formation energies and convex hulls, then display in browser
plot = binary_convex_hull_plotter(comp=comps, observed_energies=formation_energies, predicted_energies=corrs@llars_solution)
show(plot)


There are  210  nonzero ECI, of  1335  total ECI
RMSE for LassoLars:  0.0033695787446155557 eV per primitive cell


### OLS in a reduced basis vector space

Again, ordinary least squares will always find the lowest rmse solution in the given vector space. Try ordinary least squares in the vector space chosen by LassoLarsCV:

In [129]:
#Try performing ols within the reduced basis set
#Calcualte the Ordinary Least Squares solution
ols_solution = np.linalg.pinv(corrs[:,nonzero_indices].T @ corrs[:,nonzero_indices]) @ corrs[:,nonzero_indices].T @ formation_energies

#Create the convex hull plot object using bokeh
plot = binary_convex_hull_plotter(comp=comps, observed_energies=formation_energies, predicted_energies=corrs[:,nonzero_indices]@ols_solution)

print("RMSE for OLS in the LassoLars vector space (eV/primitive cell): ",mean_squared_error(formation_energies,corrs[:,nonzero_indices]@ols_solution,squared=False))

#Display the bokeh plot in browser
show(plot)


RMSE for OLS in the LassoLars vector space (eV/primitive cell):  0.003005888272026143


### Using Ground State Information

An ideal cluster expansion model will replicate observed ground states, if there are any. Ground state information can be used to guide the  model fitting process. Below are the simplest examples of this. 

In [130]:
#Calculate convex hull, hull distances and hull correlations
observed_hull = thull.full_hull(compositions=comps, energies=formation_energies)
hulldists = thull.lower_hull_distances(compositions=comps, energies=formation_energies, convex_hull=observed_hull)
hullcorr = thull.hull_distance_correlations(corrs, compositions=comps, formation_energy=formation_energies, hull=observed_hull)

#### Using ground states to truncate

Configurations with formaton energies closer to the lower convex hull are more relevant than those which are far away. This relevance follows a boltzmann distribution. 

We can bias a linear regression model to have higher accuracy on more important configurations, at the expense of lower accuracy on less important structures. 

This will reduce the overall RMSE, but allows the LassoLars regression model to more aggressively truncate the basis vector space. A user can decide  the appropriate degree of bias. 

The "intensity" of the weighting is controlled by the term "b" below. This can be thought of as an inverse temperature in the boltzmann distribution. A value of b=0 is as if there is no weighting at all. Larger values of b will make the biasing and truncation more pronounced.


For further reading: this weighting is a subset of generalized least squares, which allows a more generalized approach to weighted linear regression. 

In [131]:
#Iterate through values of the weighting parameter "b"; store rmse, model sparsity and ECIs. 
rmse_vs_b = []
b_values = np.linspace(0,100,5)
nonzero_eci_count = []
all_ecis = []

for i, b in enumerate(b_values):
    print(i)
    x_prime, y_prime = hulldist_weighting(corrs, formation_energy=formation_energies, hulldists=hulldists, b=b)
    llars_regression_object.fit(x_prime,y_prime)
    llars_solution = llars_regression_object.coef_
    rmse_vs_b.append(mean_squared_error(formation_energies,corrs@llars_solution, squared=False))
    nonzero_eci_count.append(np.count_nonzero(llars_solution))
    all_ecis.append(llars_solution)


0
1
2
3
4


In [132]:
#Visualize the model truncation and RMSE vs increasing weighting parameter b. 

s1 = figure(x_axis_type="linear", width=800, height=400)
s1.extra_y_ranges = {"rmse": Range1d(start=0, end=0.02)}
s1.circle(b_values, nonzero_eci_count, color="black",size=10, legend_label="Nonzero ECI Count")
s1.add_layout(LinearAxis(y_range_name="rmse", axis_label="RMSE", axis_label_text_color="red"), 'right')
s1.line(b_values, rmse_vs_b, line_color="red", y_range_name="rmse" , legend_label="RMSE vs. b")
s1.yaxis.axis_label = "Nonzero ECI Count"
s1.right[0].axis_label = "RMSE"

show(s1)


In [133]:
#Choose one of the truncated models
model_index_of_interest = 2
model_of_interest = all_ecis[model_index_of_interest]
print("Model of Interest RMSE (eV): ",rmse_vs_b[model_index_of_interest])
print("Nonzero ECI: ",nonzero_eci_count[model_index_of_interest])

#View the model predictions
plot = binary_convex_hull_plotter(comp=comps, observed_energies=formation_energies, predicted_energies=corrs@model_of_interest)
show(plot)

Model of Interest RMSE (eV):  0.006632645123077776
Nonzero ECI:  107


#### Using ground states to improve fit quality

The hull distances of each structure were used above to truncate a model. While this information is already contained in the correlation and formation energy data, explicitly exposing it again to a regression model can alter and potentially improve the model behavior. 

In [134]:
#Use LassoLarsCV with hull distance and hull correlations to improve fits

#Calculate convex hull, hull distances and hull correlations
observed_hull = thull.full_hull(compositions=comps, energies=formation_energies)
hulldists = thull.lower_hull_distances(compositions=comps, energies=formation_energies, convex_hull=observed_hull)
hullcorr = thull.hull_distance_correlations(corrs, compositions=comps, formation_energy=formation_energies, hull=observed_hull)

#Create a modified feature matrix and target array for regression
x_prime = np.vstack((corrs,hullcorr))
y_prime = np.vstack((formation_energies.reshape(-1,1), hulldists.reshape(-1,1)))


#Fit using the lasso lars regression object
llars_regression_object.fit(x_prime, y_prime)
llars_solution = llars_regression_object.coef_


#Observe the number of nonzero ECI:
print("There are ",np.count_nonzero(llars_solution), " nonzero ECI, of ",llars_solution.shape[0]," total ECI")

nonzero_indices = np.where(llars_solution !=0)[0]

#Check the RMSE for the current model:
print("RMSE for LassoLars: ",mean_squared_error(formation_energies,corrs@llars_solution,squared=False), "eV per primitive cell")

#Create and display a plot of the formation energies and convex hulls, then display in browser
plot = binary_convex_hull_plotter(comp=comps, observed_energies=formation_energies, predicted_energies=corrs@llars_solution)
show(plot)


/home/derick/anaconda3/envs/casmproj/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


There are  228  nonzero ECI, of  1335  total ECI
RMSE for LassoLars:  0.003181703300026893 eV per primitive cell


### Model Uncertainty: Bayesian Ridge
Most regression models produce single models. However, it is more useful to have a distribution of probable models to choose from; distributions across probable models allows for uncertainty quantification and propagation. 

It is often complicated and difficult to obtain distributions on models. A basic and effective approach is to use the Bayesian interpretation of ridge regression. 

This can be done using the BayesianRidge function from scikit learn, and is shown below. First, the bayesian ridge object is constructed, then applied to a truncated correlation basis, and observed formation energy data. 

In [135]:
#Create a Bayesian Ridge object 
bayes_ridge_object = BayesianRidge(fit_intercept=False)
bayes_ridge_object.fit(corrs[:,nonzero_indices], formation_energies)


#Display estimates for data noise and coefficient uncertainty
print("Estimated data noise, eV/primitive cell: ", np.sqrt(1/bayes_ridge_object.alpha_))
print("Estimated prior ECI uncertainty, eV: ",np.sqrt(1/bayes_ridge_object.lambda_))


#Extract the posterior mean and covariance (Bayesian Ridge is all Gaussian)
posterior_covariance_matrix = bayes_ridge_object.sigma_
posterior_mean = bayes_ridge_object.coef_

#Get a rough estimate for the posterior uncertainty on ECI values
eigvals, eigvecs = np.linalg.eig(posterior_covariance_matrix)
print("Rought estimate on posterior ECI uncertainty, eV: ",np.mean(np.sqrt(eigvals)))

#Draw 1000 samples from the posterior distribution 
samples = np.random.multivariate_normal(mean=posterior_mean, cov=posterior_covariance_matrix, size=1000)


#Visualize the distribution of the ECI

Estimated data noise, eV/primitive cell:  0.0031501760402496705
Estimated prior ECI uncertainty, eV:  0.09910966583684584
Rought estimate on posterior ECI uncertainty, eV:  0.021647865272726066
